This notebook is part of the `kikuchipy` documentation https://kikuchipy.org.
Links to the documentation won't work from the notebook.

# PC calibration: "moving-screen" technique

The projection center (PC) describing the position of the EBSD detector relative to the beam-sample interaction volume can be estimated by the "moving-screen" technique <cite data-cite="hjelen1991electron">Hjelen et al.</cite>.
In this tutorial, we test this technique to get a rough estimate of the PC.

The technique assumes that the PC vector from the detector to the sample, shown in the [top figure in the reference frames tutorial](reference_frames.ipynb#Detector-sample-geometry), is normal to the detector screen as well as the incoming electron beam.
It will therefore intersect the screen at a position independent of the detector distance (DD).
To find this position, we need two EBSD patterns acquired with a stationary beam but a known difference $\Delta z$ in DD, say 5 mm.

First, the goal is to find the pattern position that does not shift between the two camera positions, ($PC_x$, $PC_y$).
This point can be estimated in fractions of screen width and height, respectively, by selecting the same pattern features in both patterns.
The two points of each pattern feature can then be used to form a straight line, and two or more such lines should intersect at ($PC_x$, $PC_y$).

Second, the DD ($PC_z$) can be estimated from the same points.
After finding the distances $L_{in}$ and $L_{out}$ between two points (features) in both patterns (in = operating position, out = 5 mm from operating position), the DD can be found from the relation

$$
\mathrm{DD} = \frac{\Delta z}{L_{out}/L_{in} - 1},
$$

where DD is given in the same unit as $\Delta z$.
If also the detector pixel size $\delta$ is known (e.g. 46 mm / 508 px), $PC_z$ can be given in the fraction of the detector screen height

$$
PC_z = \frac{\mathrm{DD}}{N_r \delta b},
$$

where $N_r$ is the number of detector rows and $b$ is the binning factor.

Let's first import necessary libraries

In [ ]:
# Exchange inline for notebook or qt5 (from pyqt) for interactive plotting
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from diffsims.crystallography import ReciprocalLatticeVector
from orix.crystal_map import PhaseList
import kikuchipy as kp

We will find an estimate of the PC from two single crystal silicon patterns.
These are included in the [kikuchipy.data](../reference/generated/kikuchipy.data.rst) module

In [ ]:
s_in = kp.data.si_ebsd_moving_screen(0, allow_download=True)
s_in.remove_static_background("divide")
s_in.remove_dynamic_background("divide")
s_in.normalize_intensity(dtype_out="float32")

s_out5mm = kp.data.si_ebsd_moving_screen(5, allow_download=True)
s_out5mm.remove_static_background("divide")
s_out5mm.remove_dynamic_background("divide")
s_out5mm.normalize_intensity(dtype_out="float32")

As a first approximation, we can find the detector pixel positions of the same features in both patterns manually with Matplotlib.
Cursor pixel coordinates are displayed in the upper right part of the Matplotlib window when plotting with an interactive backend (e.g. qt5 or notebook).

In [ ]:
fig, axes = plt.subplots(
    ncols=2, sharex=True, sharey=True, figsize=(14, 7), layout="tight"
)
for ax, data in zip(axes, [s_in.data, s_out5mm.data]):
    ax.imshow(data, cmap="gray", vmin=-3, vmax=3)

In this example, we choose the positions of three zone axes.
The PC calibration is performed by creating an [PCCalibrationMovingScreen](../reference/generated/kikuchipy.detectors.PCCalibrationMovingScreen.rst) instance

In [ ]:
cal = kp.detectors.PCCalibrationMovingScreen(
    pattern_in=s_in.data,
    pattern_out=s_out5mm.data,
    points_in=[(109, 131), (390, 139), (246, 232)],
    points_out=[(77, 146), (424, 156), (246, 269)],
    delta_z=5,
    px_size=None,  # Default
    convention="tsl",  # Default
)
cal

We see that ($PC_x$, $PC_y$) = (0.5123, 0.8606), while DD = 21.7 mm.
To get $PC_z$ in fractions of detector height, we have to provide the detector pixel size $\delta$ upon initialization, or set it directly and recalculate the PC

In [ ]:
cal.px_size = 90e-3  # mm/px
cal

We can visualize the estimation by using the convenience method [PCCalibrationMovingScreen.plot()](../reference/generated/kikuchipy.detectors.PCCalibrationMovingScreen.plot.rst)

In [ ]:
cal.plot(pattern_kwargs={"vmin": -3, "vmax": 3, "cmap": "gray"})

As expected, the three lines in the right figure meet at approimately the same point.
We can replot the three images and zoom in on the PC to see how close they are to each other

In [ ]:
# PCy defined from top to bottom, otherwise "tsl", defined from bottom to top
cal.convention = "bruker"
pcx, pcy, _ = cal.pc

# Use two standard deviations of all $PC_x$ estimates as the axis limits
# (scaled with pattern shape)
two_std = 2 * np.std(cal.pcx_all, axis=0)

fig = cal.plot(return_figure=True)
ax2 = fig.axes[2]
ax2.set_xlim([cal.ncols * (pcx - two_std), cal.ncols * (pcx + two_std)])
ax2.set_ylim([cal.nrows * (pcy - two_std), cal.nrows * (pcy + two_std)])
fig.subplots_adjust(wspace=0.05)

We can use this PC estimate as an initial guess when refining the PC using Hough indexing available from [PyEBSDIndex](https://pyebsdindex.readthedocs.io).
See the [Hough indexing tutorial](hough_indexing.ipynb) for details.

<div class="alert alert-info">

Note

PyEBSDIndex is an optional dependency of kikuchipy, and can be installed with both `pip` and `conda` (from `conda-forge`).
To install PyEBSDIndex, see their [installation instructions](https://pyebsdindex.readthedocs.io/en/latest/user/installation.html).

</div>

Create a detector with the correct shape and sample tilt, adding the PC estimate

In [ ]:
det = kp.detectors.EBSDDetector(cal.shape, sample_tilt=70, pc=cal.pc)
det

Create an EBSDIndexer for use with PyEBSDIndex

In [ ]:
phase_list = PhaseList(names="si", space_groups=227)
print(phase_list)

indexer = det.get_indexer(phase_list)

Optimize the PC via Hough indexing and plot the difference between the estimated and optimized PCs

In [ ]:
det_ref = s_in.hough_indexing_optimize_pc(
    pc0=det.pc, indexer=indexer, method="PSO"
)

print(abs(det.pc - det_ref.pc))

Index the pattern via Hough indexing

In [ ]:
indexer.PC = det_ref.pc

xmap = s_in.hough_indexing(phase_list, indexer=indexer, verbose=2)

In [ ]:
print(xmap)
print(xmap.fit)

Create a simulator with the five $\{hkl\}$ families (reflectors) $\{111\}$, $\{200\}$, $\{220\}$, $\{222\}$, and $\{311\}$

In [ ]:
ref = ReciprocalLatticeVector(
    phase=xmap.phases[0],
    hkl=[[1, 1, 1], [2, 0, 0], [2, 2, 0], [2, 2, 2], [3, 1, 1]],
)
ref = ref.symmetrise()

simulator = kp.simulations.KikuchiPatternSimulator(ref)

sim = simulator.on_detector(det_ref, xmap.rotations)

Plot a [geometrical simulation](geometrical_ebsd_simulations.ipynb) on top of the pattern using Kikuchi band centers

In [ ]:
sim.plot(
    coordinates="gnomonic",
    pattern=s_in.data,
    zone_axes_labels=False,
    zone_axes=False,
    pattern_kwargs={"vmin": -3, "vmax": 3},
)